**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Linux namespaces

![Namespaces](namespaces.png)

![container vs hypervisor](docker-hypervisor.png)

In [1]:
docker ps

CONTAINER ID   IMAGE                  COMMAND                  CREATED          STATUS          PORTS     NAMES
012ab7232469   86262685d9ab           "/metrics-sidecar"       22 minutes ago   Up 22 minutes             k8s_dashboard-metrics-scraper_dashboard-metrics-scraper-f6647bd8c-cst2g_kubernetes-dashboard_609139cd-a00f-444a-927a-cb60657ef7d2_0
a916f22652df   9a07b5b4bfac           "/dashboard --insecu…"   22 minutes ago   Up 22 minutes             k8s_kubernetes-dashboard_kubernetes-dashboard-968bcb79-zzwcl_kubernetes-dashboard_ea46ab81-d0df-4bcd-9e23-f8a998ca2cc9_0
14d3c766e0d6   k8s.gcr.io/pause:3.2   "/pause"                 22 minutes ago   Up 22 minutes             k8s_POD_dashboard-metrics-scraper-f6647bd8c-cst2g_kubernetes-dashboard_609139cd-a00f-444a-927a-cb60657ef7d2_0
b70831264217   k8s.gcr.io/pause:3.2   "/pause"                 22 minutes ago   Up 22 minutes             k8s_POD_kubernetes-dashboard-968bcb79-zzwcl_kubernetes-dashboard_ea46ab81-d0df-4bcd-9e23-f8a998ca2cc9

In [2]:
crictl ps

CONTAINER           IMAGE                                                                           CREATED             STATE               NAME                        ATTEMPT             POD ID
012ab72324693       86262685d9abb                                                                   23 minutes ago      Running             dashboard-metrics-scraper   0                   14d3c766e0d6c
a916f22652dfc       9a07b5b4bfac0                                                                   23 minutes ago      Running             kubernetes-dashboard        0                   b708312642179
193f7d2f5f9be       nginx@sha256:75a55d33ecc73c2a242450a9f1cc858499d468f077ea942867e662c247b5e412   36 minutes ago      Running             my-nginx                    0                   792d59f591baf
0634591360c48       nginx@sha256:75a55d33ecc73c2a242450a9f1cc858499d468f077ea942867e662c247b5e412   36 minutes ago      Running             my-nginx                    0                   032d8ef380d

In [12]:
POD=k8s_my-nginx_my-nginx-5b56ccd65f-zl5ss_default_61f63d48-2331-4aba-a4ff-2266854a1aa7_0

In [13]:
STATE_PID=$(docker inspect --format '{{.State.Pid}}' $POD)
echo Enter
echo nsenter -t $STATE_PID -n bash
echo in a terminal  

Enter
nsenter -t 4932 -n bash
in a terminal


In [5]:
nsenter --help


Usage:
 nsenter [options] [<program> [<argument>...]]

Run a program with namespaces of other processes.

Options:
 -a, --all              enter all namespaces
 -t, --target <pid>     target process to get namespaces from
 -m, --mount[=<file>]   enter mount namespace
 -u, --uts[=<file>]     enter UTS namespace (hostname etc)
 -i, --ipc[=<file>]     enter System V IPC namespace
 -n, --net[=<file>]     enter network namespace
 -p, --pid[=<file>]     enter pid namespace
 -C, --cgroup[=<file>]  enter cgroup namespace
 -U, --user[=<file>]    enter user namespace
 -S, --setuid <uid>     set uid in entered namespace
 -G, --setgid <gid>     set gid in entered namespace
     --preserve-credentials do not touch uids or gids
 -r, --root[=<dir>]     set the root directory
 -w, --wd[=<dir>]       set the working directory
 -F, --no-fork          do not fork before exec'ing <program>
 -Z, --follow-context   set SELinux context according to --target PID

 -h, --help             display this help
 -V

In [5]:
nsenter -t $STATE_PID -n ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
5: eth0@if6: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:03 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.3/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [6]:
ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:11:8d:eb:29 brd ff:ff:ff:ff:ff:ff
    inet 172.17.0.1/16 brd 172.17.255.255 scope global docker0
       valid_lft forever preferred_lft forever
4: vetha7dd9b9@if3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether 46:50:36:75:89:d2 brd ff:ff:ff:ff:ff:ff link-netnsid 1
6: vethff9f8d5@if5: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether 92:9a:3f:b8:2d:62 brd ff:ff:ff:ff:ff:ff link-netnsid 2
8: veth4aa3c4f@if7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether a2:5b:0

 # Danger!
 
 OpenVpn works and could be used as backdoor

In [7]:
nsenter -t $STATE_PID -a hostname

nsenter: failed to execute hostname: No such file or directory


: 127

In [8]:
nsenter -t $STATE_PID -a ip a

nsenter: failed to execute ip: No such file or directory


: 127

In [9]:
nsenter -t $STATE_PID -n ip route

default via 172.17.0.1 dev eth0 
172.17.0.0/16 dev eth0 proto kernel scope link src 172.17.0.3 


In [10]:
ip route

default via 192.168.49.1 dev eth0 
172.17.0.0/16 dev docker0 proto kernel scope link src 172.17.0.1 
192.168.49.0/24 dev eth0 proto kernel scope link src 192.168.49.2 


In [14]:
nsenter -t $STATE_PID -a ls

bin   docker-entrypoint.d   home   media  proc	sbin  tmp
boot  docker-entrypoint.sh  lib    mnt	  root	srv   usr
dev   etc		    lib64  opt	  run	sys   var


In [15]:
ls

 AccountWithRBAC.ipynb       NetworkPolicy.ipynb
 AuditLogs.ipynb             OpenPolicyAgent.ipynb
 ClusterOverview.ipynb       Overview.ipynb
 ContainerdCloudbomb.ipynb   PodSecurityPolicyBasic.ipynb
 CrioBomb.ipynb              PodSecurityPolicyPermissive.ipynb
 Harbor.ipynb                PodSecurityPolicyStrict.ipynb
 Helm.ipynb                 'Postgres Zalando.ipynb'
 Ingress.ipynb               README.md
 IngressStart.ipynb          ServiceAccountToken.ipynb
 Istio-Traffic-Mgmt.ipynb    Unprivileged.ipynb
 Istio1.8.2.ipynb            default-rbac.png
 IstioHack.ipynb             defaut-rbac.dot
 Kube-Scan-Cloudbomb.ipynb   docker-hypervisor.png
 KubernetesBasics.ipynb      kube-bench.ipynb
 License.ipynb               minikube.dot
'Linux Namespaces.ipynb'     my-namespace-rbac.dot
 Makefile                    namespaces.dot


In [16]:
nsenter -t $STATE_PID -a ps auxwww

nsenter: failed to execute ps: No such file or directory


: 127

In [17]:
ps auxww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  22020 11624 ?        Ss   09:07   0:00 /sbin/init
root         168  0.0  0.0  29076 12264 ?        S<s  09:07   0:00 /lib/systemd/systemd-journald
message+     180  0.0  0.0   7008  3728 ?        Ss   09:07   0:00 /usr/bin/dbus-daemon --system --address=systemd: --nofork --nopidfile --systemd-activation --syslog-only
root         186  0.2  0.0 2154852 52932 ?       Ssl  09:07   0:06 /usr/bin/containerd
root         196  0.0  0.0  12184  7288 ?        Ss   09:07   0:00 sshd: /usr/sbin/sshd -D [listener] 0 of 10-100 startups
root         218  0.3  0.1 337100 76436 ?        Ssl  09:07   0:09 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root         222  1.5  0.1 2424816 117316 ?      Ssl  09:07   0:45 /usr/bin/dockerd -H tcp://0.0.0.0:2376 -H unix:///var/run/docker.sock --default-ulimit=nofile=1048576:1048576 --tlsverify --tlscacert /etc/doc

In [19]:
nsenter -t $STATE_PID -pm ps auxwww

nsenter: cannot open m: No such file or directory


: 1